Disclaimer - Use results and my commentary at your own risk - I am not a medical professional or a health statistician.

This is just me playing with some data and creating stories from it.  I hope you enjoy and learn something.

All data from https://covidtracking.com/data

Data ran in this notebook is current as of June 1, 2020.

Create a local path to download the csv data to

In [4]:
localpath="dbfs:/USACovid19/"

##Remove directory
dbutils.fs.rm(localpath,True)

Out[21]: True

In [5]:
dbutils.fs.mkdirs(localpath)



Out[22]: True

Download COVID19 data

In [7]:
%sh
wget -P /dbfs/USACovid19/ https://covidtracking.com/api/v1/states/daily.csv

--2020-06-02 21:15:52-- https://covidtracking.com/api/v1/states/daily.csv
Resolving covidtracking.com (covidtracking.com)... 162.243.166.170, 2604:a880:400:d1::888:7001
Connecting to covidtracking.com (covidtracking.com)|162.243.166.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 986984 (964K) [text/csv]
Saving to: ‘/dbfs/USACovid19/daily.csv’

 0K .......... .......... .......... .......... .......... 5% 3.61M 0s
 50K .......... .......... .......... .......... .......... 10% 23.0M 0s
 100K .......... .......... .......... .......... .......... 15% 29.1M 0s
 150K .......... .......... .......... .......... .......... 20% 27.8M 0s
 200K .......... .......... .......... .......... .......... 25% 15.2M 0s
 250K .......... .......... .......... .......... .......... 31% 7.66M 0s
 300K .......... .......... .......... .......... .......... 36% 8.75M 0s
 350K .......... .......... .......... .......... .......... 41% 8.41M 0s
 400K .......... .......... .......... .......... .......... 46% 8.13M 0s
 450K .......... .......... .......... .......... .......... 51% 22.6M 0s
 500K .......... .......... .......... .......... .......... 57% 8.66M 0s
 550K .......... .......... .......... .......... .......... 62% 8.33M 0s
 600K .......... .......... .......... .......... .......... 67% 7.19M 0s
 650K .......... .......... .......... .......... .......... 72% 25.6M 0s
 700K .......... .......... .......... .......... .......... 77% 8.77M 0s
 750K .......... .......... .......... .......... .......... 83% 7.17M 0s
 800K .......... .......... .......... .......... .......... 88% 7.47M 0s
 850K .......... .......... .......... .......... .......... 93% 8.35M 0s
 900K .......... .......... .......... .......... .......... 98% 14.8M 0s
 950K .......... ... 100% 25.0M=0.1s

2020-06-02 21:15:52 (9.70 MB/s) - ‘/dbfs/USACovid19/daily.csv’ saved [986984/986984]

In [8]:
dbutils.fs.ls ('dbfs:/USACovid19')

Out[24]: [FileInfo(path='dbfs:/USACovid19/daily.csv', name='daily.csv', size=986984)]

Create a Dataframe to hold COVID19 data for all US states reporting

In [10]:
dfUSACovid19 = (
  spark
  .read
  .option ("header",True)
  .option ("inferschema",True)
  .csv("dbfs:/USACovid19/daily.csv")
      )

In [11]:
display(dfUSACovid19).take(20)

date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
20200602,AK,487,55716,null,10,null,null,null,0.0,null,371,A,6/2/2020 00:00,2020-06-02T00:00:00.000+0000,06/01 20:00,10,null,2020-06-02T00:00:00.000+0000,2,20,1993,56203,56203,2013,56203,0,0,8bfc576f3ce0c5438919ab2a672255d0630714ec,0,0,0,0,0,null
20200602,AL,18642,205160,null,null,1879,null,591,null,355,9355,B,6/1/2020 00:00,2020-06-01T00:00:00.000+0000,05/31 20:00,651,1879,2020-06-01T00:00:00.000+0000,1,279,0,223802,223802,279,223802,5,23,e925d317dd6bba774728abaf5db247f3c5983394,0,0,0,0,0,null
20200602,AR,7443,125793,null,121,718,null,null,26.0,124,5401,A,6/2/2020 00:00,2020-06-02T00:00:00.000+0000,06/01 20:00,133,718,2020-06-02T00:00:00.000+0000,5,0,0,133236,133236,0,133236,0,7,1456f854de8dbd854d74cf88314b9c36bdf5c78a,0,0,0,0,0,null
20200602,AS,0,174,null,null,null,null,null,null,null,null,C,5/29/2020 00:00,2020-05-29T00:00:00.000+0000,05/28 20:00,0,null,2020-05-29T00:00:00.000+0000,60,0,0,174,174,0,174,0,0,cccae3dfdebc5c6a2b53fdb498e0fbce6aec0120,0,0,0,0,0,null
20200602,AZ,21250,216583,null,1009,3057,380,null,235.0,null,4967,A+,6/2/2020 00:00,2020-06-02T00:00:00.000+0000,06/01 20:00,941,3057,2020-06-02T00:00:00.000+0000,4,1127,8636,237833,237833,9763,237833,24,39,40c3b8895be0300a58790c838cb9a3a154d62cc0,0,0,0,0,0,null
20200602,CA,115310,1956281,null,4393,null,1286,null,null,null,null,B,6/2/2020 00:00,2020-06-02T00:00:00.000+0000,06/01 20:00,4286,null,2020-06-02T00:00:00.000+0000,6,2304,56704,2071591,2071591,59008,2071591,35,0,9c03c1926787907836d6c35b13c8e44c97dc26b7,0,0,0,0,0,null
20200602,CO,26577,163960,null,394,4372,null,null,null,null,3954,A,6/2/2020 00:00,2020-06-02T00:00:00.000+0000,06/01 20:00,1458,4372,2020-06-02T00:00:00.000+0000,8,199,3962,190537,190537,4161,190537,13,25,0a4c8534ed6ced43b0835e9e7da33d18914d4141,0,0,0,0,0,null
20200602,CT,42979,227631,null,434,13793,null,null,null,null,7511,B,6/1/2020 20:30,2020-06-01T20:30:00.000+0000,06/01 16:30,3972,13793,2020-06-01T20:30:00.000+0000,9,239,8981,270610,270610,9220,270610,8,0,d21862626f593d49b27bfa13773b02a3eebcd900,0,0,0,0,0,null
20200602,DC,8886,38815,null,295,null,106,null,74.0,null,1137,A+,6/1/2020 00:00,2020-06-01T00:00:00.000+0000,05/31 20:00,470,null,2020-06-01T00:00:00.000+0000,11,29,409,47701,47701,438,47701,2,0,36f51c85e83d22ab21e78b35a025b3314b2be260,0,0,0,0,0,null
20200602,DE,9685,54377,null,157,null,null,null,null,null,5442,A,6/1/2020 18:00,2020-06-01T18:00:00.000+0000,06/01 14:00,373,null,2020-06-01T18:00:00.000+0000,10,80,1535,64062,64062,1615,64062,5,0,a6601e4f641beae62fccabfc1cf8aa554f65f260,0,0,0,0,0,null


For my story, I chose to play with State of Maryland Data

Creating a new dataframe for only MD data from March 29, 2020

In [14]:
##State of Maryland didn't report negatives between 3/12 and 3/28 so data will be used starting 3/29/2020

from pyspark.sql import *

dfMDCovid19 = dfUSACovid19.filter((dfUSACovid19.state == "MD") & (dfUSACovid19.date >= "20200329")).sort(dfUSACovid19.date)

display(dfMDCovid19)

date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
20200329,MD,1239,12354,null,null,277,null,null,null,null,null,D,3/29/2020 10:00,2020-03-29T10:00:00.000+0000,03/29 06:00,10,277,2020-03-29T10:00:00.000+0000,24,247,838,13593,13593,1085,13593,5,51,99c2a57a9870ca53eb8e08e49afad92542cb62e5,0,0,0,0,0,null
20200330,MD,1413,13316,null,null,353,null,null,null,null,null,A,3/30/2020 10:00,2020-03-30T10:00:00.000+0000,03/30 06:00,15,353,2020-03-30T10:00:00.000+0000,24,174,962,14729,14729,1136,14729,5,76,5666d5f891c2c7bb73cb33020917d93fd2706b6e,0,0,0,0,0,null
20200331,MD,1660,14868,null,null,429,null,null,null,null,null,A,3/31/2020 10:00,2020-03-31T10:00:00.000+0000,03/31 06:00,18,429,2020-03-31T10:00:00.000+0000,24,247,1552,16528,16528,1799,16528,3,76,d6eaae443a0b9bf17605c9f8b91cde22f1525ea7,0,0,0,0,0,null
20200401,MD,1985,17233,null,null,522,null,null,null,null,null,A,4/1/2020 10:00,2020-04-01T10:00:00.000+0000,04/01 06:00,31,522,2020-04-01T10:00:00.000+0000,24,325,2365,19218,19218,2690,19218,13,93,dc24f941a30f7b77f2c42b02918167150bedc6df,0,0,0,0,0,null
20200402,MD,2331,18890,null,null,582,null,null,null,null,81,A,4/2/2020 10:00,2020-04-02T10:00:00.000+0000,04/02 06:00,36,582,2020-04-02T10:00:00.000+0000,24,346,1657,21221,21221,2003,21221,5,60,2524161fea18dbe78f40d2f095c9dc1a5b515637,0,0,0,0,0,null
20200403,MD,2758,20932,null,null,664,null,null,null,null,159,A,4/3/2020 10:00,2020-04-03T10:00:00.000+0000,04/03 06:00,42,664,2020-04-03T10:00:00.000+0000,24,427,2042,23690,23690,2469,23690,6,82,7346ddbf0fee002be1916f6e12661ce264310c3d,0,0,0,0,0,null
20200404,MD,3125,22485,null,null,821,null,null,null,null,159,A,4/4/2020 10:00,2020-04-04T10:00:00.000+0000,04/04 06:00,53,821,2020-04-04T10:00:00.000+0000,24,367,1553,25610,25610,1920,25610,11,157,7b5bd8eb0e76c1abb2776ce773247a13f870f250,0,0,0,0,0,null
20200405,MD,3609,24728,null,null,936,null,null,null,null,159,A,4/5/2020 10:00,2020-04-05T10:00:00.000+0000,04/05 06:00,67,936,2020-04-05T10:00:00.000+0000,24,484,2243,28337,28337,2727,28337,14,115,636737bab78b465c34b5f9f3ef456759243031f2,0,0,0,0,0,null
20200406,MD,4045,25572,null,null,1059,null,null,null,null,184,A,4/6/2020 10:00,2020-04-06T10:00:00.000+0000,04/06 06:00,91,1059,2020-04-06T10:00:00.000+0000,24,436,844,29617,29617,1280,29617,24,123,f6ce757f3ddee0d306384d868a1a0fc678e02c05,0,0,0,0,0,null
20200407,MD,4371,27256,null,null,1106,null,null,null,null,288,A,4/7/2020 10:00,2020-04-07T10:00:00.000+0000,04/07 06:00,103,1106,2020-04-07T10:00:00.000+0000,24,326,1684,31627,31627,2010,31627,12,47,1e64810c79e672682e5a8e0b7160310e3261aa76,0,0,0,0,0,null


Creating a view from the Dataframe so I can run SQL queries against it

In [16]:
dfMDCovid19.createOrReplaceTempView("vw_SQLMDCovid19")

Ordered by the date with the highest percent positive rating (for cases) descending

In [18]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by PercentPositive desc

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200413,1987,711,1276,35.782586814292905,64.2174131857071
20200406,1280,436,844,34.0625,65.9375
20200508,3587,1111,2476,30.972957903540564,69.02704209645944
20200415,2030,560,1470,27.586206896551722,72.41379310344827
20200509,3841,1049,2792,27.310596198906534,72.68940380109346
20200414,1982,536,1446,27.04339051463169,72.95660948536832
20200510,3903,1053,2850,26.979246733282093,73.0207532667179
20200423,3571,962,2609,26.939232707924948,73.06076729207504
20200410,2919,783,2136,26.82425488180884,73.17574511819116
20200506,3909,1046,2863,26.7587618316705,73.24123816832949


Ordered by number of Positive Cases

In [20]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by positiveIncrease desc

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200519,7152,1784,5368,24.94407158836689,75.0559284116331
20200501,6624,1730,4894,26.117149758454104,73.8828502415459
20200528,12803,1286,11517,10.044520815433883,89.95547918456612
20200529,9660,1279,8381,13.24016563146998,86.75983436853002
20200507,4588,1211,3377,26.3949433304272,73.6050566695728
20200521,4903,1208,3695,24.637976748929226,75.36202325107078
20200408,6835,1158,5677,16.942209217264082,83.05779078273592
20200425,4407,1150,3257,26.094849103698657,73.90515089630134
20200508,3587,1111,2476,30.972957903540564,69.02704209645944
20200514,4880,1091,3789,22.356557377049178,77.64344262295081


Ordered by Date (latest first)

In [22]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by date desc

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200602,7085,848,6237,11.968948482709951,88.03105151729005
20200601,6849,549,6300,8.015768725361367,91.98423127463863
20200531,7935,763,7172,9.615626969124133,90.38437303087586
20200530,9428,1027,8401,10.893084429359355,89.10691557064064
20200529,9660,1279,8381,13.24016563146998,86.75983436853002
20200528,12803,1286,11517,10.044520815433883,89.95547918456612
20200527,7568,736,6832,9.725158562367865,90.27484143763213
20200526,4910,535,4375,10.896130346232178,89.10386965376782
20200525,9215,839,8376,9.104720564297342,90.89527943570266
20200524,8035,818,7217,10.180460485376477,89.81953951462353


Percentage of positive cases reported - as per the article quoted in my blog post it looks like it's going down recently, about a half or even a one third of what is was in the higher months (April/May)

In [24]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by date

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200329,1085,247,838,22.764976958525345,77.23502304147466
20200330,1136,174,962,15.316901408450704,84.6830985915493
20200331,1799,247,1552,13.729849916620346,86.27015008337966
20200401,2690,325,2365,12.0817843866171,87.9182156133829
20200402,2003,346,1657,17.274088866699948,82.72591113330004
20200403,2469,427,2042,17.294451194815714,82.70554880518428
20200404,1920,367,1553,19.114583333333332,80.88541666666667
20200405,2727,484,2243,17.74844151081775,82.25155848918226
20200406,1280,436,844,34.0625,65.9375
20200407,2010,326,1684,16.218905472636816,83.78109452736318


The chart below shows a different story.  This is the number of cases reported per day, against the number of total tests done for the particular day. It is a numbers to numbers comparison; unlike the comparison made on Maryland's official website which shows a percentage of positive cases against number of total tests administered.

As you can see, over time, the state received more and more test results and the percentage (above) of positive cases has gone down.  This is not an indication of cases going down however.  If you think about it, just because you had a lower percentage, doesn't necessarily mean that cases are going down; your test population was lower.  Unless you test everyone everyday, it's like comparing apples to oranges; yes they are both fruit but each is still different than the other.

For example, if you look at the raw data you can start to see how statistics can tell a different story based on how you use them:
(Based on Data from June 2, 2020 run)

April 13, 2020 - # of Positive Cases 711 vs Percentage of Positive Cases ~36%

May 19, 2020 - # of Positive Cases 1784 vs Percentage of Positive Cases ~25%

May 28, 2020 - # of Positive Cases 1286 vs Percentage of Positive Cases ~10%

Are there really 10% fewer cases between April 3th and May 19th?

Was there a drop of 72% in positive cases between April 13 and May 28th?

It is very hard to determine that just based on percentages, because we don't have a equal sample of test cases on both days.  Plus how many of those recovered, how many of those got reinfect and retested.  Who knows???

In [26]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by date

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200329,1085,247,838,22.764976958525345,77.23502304147466
20200330,1136,174,962,15.316901408450704,84.6830985915493
20200331,1799,247,1552,13.729849916620346,86.27015008337966
20200401,2690,325,2365,12.0817843866171,87.9182156133829
20200402,2003,346,1657,17.274088866699948,82.72591113330004
20200403,2469,427,2042,17.294451194815714,82.70554880518428
20200404,1920,367,1553,19.114583333333332,80.88541666666667
20200405,2727,484,2243,17.74844151081775,82.25155848918226
20200406,1280,436,844,34.0625,65.9375
20200407,2010,326,1684,16.218905472636816,83.78109452736318


In [27]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by date

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200329,1085,247,838,22.764976958525345,77.23502304147466
20200330,1136,174,962,15.316901408450704,84.6830985915493
20200331,1799,247,1552,13.729849916620346,86.27015008337966
20200401,2690,325,2365,12.0817843866171,87.9182156133829
20200402,2003,346,1657,17.274088866699948,82.72591113330004
20200403,2469,427,2042,17.294451194815714,82.70554880518428
20200404,1920,367,1553,19.114583333333332,80.88541666666667
20200405,2727,484,2243,17.74844151081775,82.25155848918226
20200406,1280,436,844,34.0625,65.9375
20200407,2010,326,1684,16.218905472636816,83.78109452736318


In [28]:
%sql
Select date,totalTestResultsIncrease,positiveIncrease,negativeIncrease, 
positiveIncrease/totalTestResultsIncrease*100 as PercentPositive, negativeIncrease/totalTestResultsIncrease*100 as PercentNegative
from vw_SQLMDCovid19
order by date

date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,PercentPositive,PercentNegative
20200329,1085,247,838,22.764976958525345,77.23502304147466
20200330,1136,174,962,15.316901408450704,84.6830985915493
20200331,1799,247,1552,13.729849916620346,86.27015008337966
20200401,2690,325,2365,12.0817843866171,87.9182156133829
20200402,2003,346,1657,17.274088866699948,82.72591113330004
20200403,2469,427,2042,17.294451194815714,82.70554880518428
20200404,1920,367,1553,19.114583333333332,80.88541666666667
20200405,2727,484,2243,17.74844151081775,82.25155848918226
20200406,1280,436,844,34.0625,65.9375
20200407,2010,326,1684,16.218905472636816,83.78109452736318


Look at this chart, this is a percentage of positive vs negative cases (percent) but it is over the TOTAL aggregate over time (running total).

It shows that for the last two months, positive cases have been around the 17-20% range. If you go back to March, that range gets wider to 9-20%.

Again, another way to tell a different story with the same data.

I love this bar graph, because it shows that although Negative results are increasing, Positive are not really decreasing but they are increasing also.

Just look at the raw numbers.

April 13, 2020 - 8936 total positive (cumulative)  - 42815 total negative (cumulative)  - total cumulative 51751 - percent positive ~17.27%

June 1, 2020 - 53327 total positive (cumulative) - 255403 total negative (cumulative) - total cumulative 308730 - percent positive ~17.27%

In [31]:
%sql
Select date,totalTestResults,positive,negative,
positive/totalTestResults*100 as PercentPositive, negative/totalTestResults*100 as PercentNegative
from vw_SQLMDCovid19
order by date

date,totalTestResults,positive,negative,PercentPositive,PercentNegative
20200329,13593,1239,12354,9.114985654380932,90.88501434561907
20200330,14729,1413,13316,9.593319302057166,90.40668069794283
20200331,16528,1660,14868,10.043562439496611,89.95643756050339
20200401,19218,1985,17233,10.328858361952337,89.67114163804767
20200402,21221,2331,18890,10.984402243061119,89.01559775693887
20200403,23690,2758,20932,11.642043056141832,88.35795694385816
20200404,25610,3125,22485,12.202264740335806,87.7977352596642
20200405,28337,3609,24728,12.735998870734374,87.26400112926562
20200406,29617,4045,25572,13.65769659317284,86.34230340682716
20200407,31627,4371,27256,13.820469851708983,86.17953014829102


Like the chart above, this one gives you the percentage of Positive from the cumulative number of tests and the same percentage for negative. Pretty much what I explained above.  You can see from the graph, there has not been much change.

In [33]:
%sql
Select date,totalTestResults,positive,negative, 
positive/totalTestResults*100 as PercentPositive, negative/totalTestResults*100 as PercentNegative
from vw_SQLMDCovid19
order by date

date,totalTestResults,positive,negative,PercentPositive,PercentNegative
20200329,13593,1239,12354,9.114985654380932,90.88501434561907
20200330,14729,1413,13316,9.593319302057166,90.40668069794283
20200331,16528,1660,14868,10.043562439496611,89.95643756050339
20200401,19218,1985,17233,10.328858361952337,89.67114163804767
20200402,21221,2331,18890,10.984402243061119,89.01559775693887
20200403,23690,2758,20932,11.642043056141832,88.35795694385816
20200404,25610,3125,22485,12.202264740335806,87.7977352596642
20200405,28337,3609,24728,12.735998870734374,87.26400112926562
20200406,29617,4045,25572,13.65769659317284,86.34230340682716
20200407,31627,4371,27256,13.820469851708983,86.17953014829102
